# Deep learning pour le texte
L'objectif de ce TP est d'apprivoiser les embeddings de mots, de tokens, ainsi que les architectures Transformer. Voici ce que nous allons aborder : 
- Chargement des embeddings pré-entrainés
- Manipulation des embeddings
- Visualuation des embeddings
- Classification de phrases courtes, en fonction de différentes représentations et architectures
- Classification de textes : Finetuning d'un Transformer pre-entraîné sur le corpus IMDB 

Vous aurez besoin des installations suivantes (en plus de pytorch et numpy): 
- conda install pandas
- conda install matplotlib
- conda install -c anaconda scikit-learn
- conda install ipywidgets
- conda install gensim
- conda install datasets 
- conda install -c huggingface transformers
- conda install tqdm




 ## Configuration environnement

In [ ]:
import torch

# pour colab, decommenter:    
# !pip install torch==1.8.0+cu111 torchtext==0.9.0 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

print(torch.__version__)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

# Progrès
from tqdm import tqdm
from tqdm.autonotebook import tqdm

import pandas as pd
import numpy as np

import transformers
print(transformers.__version__)

## Chargement d'embeddings et calculs de similarité

Pour commencer à travailler sur le texte, nous allons dans un premier temps utiliser un ensemble de vecteurs d'embeddings appris via le modèle Glove (similaire à Word2Vec vu en cours), que nous allons charger via la librairie gensim (il s'agit d'un ensemble d'embeddings parmi d'autres, de nombreux autres, appris sur des corpus différents existent dans gensim ou d'autres librairies comme spacy). 

In [ ]:
import gensim.downloader as api
embeds = api.load("glove-wiki-gigaword-50")

# affichage de l'embedding du mot "book"
print(embeds['book']) 




In [ ]:
# Methode most_similar permet d'obtenir les mots les plus proches d'un mot dans l'espace d'embeddings. sa specification peut être obtenue en executant la ligne suivante : 
help(embeds.most_similar) 


Afficher les 5 mots les plus proches de "cat" via la méthode model.most_similar

In [ ]:
#[[Student/]]

#[[/Student]]


In [ ]:
# La similarité est calculée selon une mesure de cosinus dans l'espace des embeddings
# par exemple
print(embeds.similarity("apple", "banana"))
print(embeds.similarity("apple", "dog"))

### Out Of Distribution 
Attention, bien sûr tous les mots possibles ne sont pas inclus dans le dictionnaire d'embeddings, leur sémantique dépend notamment fortement du corpus sur lequel ils ont été appris.

Par exemple le mot "covid" n'est pas présent, l'execution de model['covid'] ferait planter l'execution.

Pour éviter ce genre de problème par exemple pour le traitement d'un texte ne contenant pas certains mots, ils convient de vérifier leur présence dans le vocabulaire (et alors ignorer les mots correspondants). Ceci peut se faire via vocab.keys comme ci-dessous. 

In [ ]:
# Le vocabulaire du modèle peut être obtenus via vocab.keys, qui retourne l'ensemble des tokens (mots) pour lesquels il existe un embedding dans le modèle
vocab = embeds.key_to_index.keys()
np.random.choice(list(vocab), 5)

x="covid"
if x in vocab:
    print(embeds[x])
else: print("oov")



### Visualisation des embeddings

On souhaite maintenant visualiser les embeddings dans un espace en 2D. Pour cela, on utilise t-SNE.

In [ ]:
words=['cat','mouse','dog','car','truck','motorcycle','bike','lion','plane','deers']

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def tsne_plot(model, words,n_components=2,perplexity=40):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []
    vocab = model.key_to_index.keys()
    
    for word in words:
        if word in vocab: 
            tokens.append(model[word])
            labels.append(word)

    tsne_model = TSNE(perplexity=perplexity, n_components=n_components, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)
    fig = plt.figure(figsize=(16, 16))
    if n_components==3:
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(new_values[:,0],new_values[:,1],new_values[:,2],c="r",marker="o")
        for i in range(len(new_values)):
            ax.text(new_values[i][0],new_values[i][1],new_values[i][2],labels[i])
    else:
        plt.scatter(new_values[:,0],new_values[:,1])
        for i in range(len(new_values)):
            plt.annotate(labels[i],
                        xy=(new_values[i][0],new_values[i][1]),
                        xytext=(5, 2),
                        textcoords='offset points',
                        ha='right',
                        va='bottom')
    return new_values,labels

new_values,labels = tsne_plot(embeds,words,n_components=2,perplexity=2)

### Index des Embeddings

Lors de l'établissement de modèles utilisant les embeddings du vocabulaire, il est utile de considérer des indices des mots dans le vocabulaire plutôt que les mots eux mêmes (pour les traiter par exemple dans des tenseurs avant de les remplacer par leurs vecteurs de poids). Ceci se fait dans gensim par l'utilisation des instructions suivantes:  

In [ ]:
print(embeds.key_to_index["book"])
print(embeds.index_to_key[539])

# Deux manières equivalentes de recupérer les poids
print(embeds["book"])   # via le mot 
print(embeds.vectors[embeds.key_to_index["book"]]) #via l'index


# taille du vocabulaire: 
print(embeds.vectors.shape)


#### Exercice 1 : Similarité de phrases

On souhaite calculer la matrice de similarité de différentes phrases du dataset suivant:




In [ ]:
tdf = pd.DataFrame([
    ['the road is straight', 'Y'],
    ['the black cat plays with a ball', 'N'],
    ['a big dog with a ball', 'N'],
    ['dog and cat are together', 'N'],
    ['traffic jam on the 6th road', 'Y'],
    ['white bird on a big tree', 'N'],
    ['a big truck', 'Y'],
    ['two cars crashed', 'Y'],
    ['two deers in a field', 'N'],
    ['I like ridding my bike','Y'],
    ['a lion in the savane','N'],
    ['a motorcycle rides on the road','Y'],
    ['it is a bike, it is not a flamingo', 'Y'], 
    ['it is not a bike, it is a flamingo', 'N'],
    ['a mouse bitten by a cat','N'],
    ['two pigs in the mood','N'],
    ['take a plane is sometimes slower than taking train','Y'],
    ['take the highway','Y']
], columns=['text', 'label'])
tdf

#
#  
#

Pour chaque phrase, on va moyenner les embeddings de ces mots. Chaque vecteur de phrase sera normalisé (x.norm()). Les phrases pourront être comparées deux à deux par un produit scalaire.

- chaque phrase doit être découpée : ("the", "cat", "is" , "on", "the","bank")
- on définit une fonction getvectors qui à partir de la liste de phrases découpées : moyenne les vecteurs d'embeddings (pensez à utiliser les tenseurs de torch), normalise le resultat et le retourne. 
- on peut ensuite calculer la matrice de similarité qui sera donnée à la fonction visual_similarity_matrix() fournie ci-dessous.

In [ ]:

# Premiere possibilité :
def getlistwordsentence(text):
    ret=[]
    for sentence in text:
        print(sentence)
        ret.append(sentence.split(" "))
    return ret


# Deuxième possibilité (plus robuste, gère la ponctuation, etc.):
import gensim
def getlistwordsentence2(text):
    ret=[]
    for sentence in text:
        ret.append(list(gensim.utils.tokenize(sentence)))
    return ret

text_wordlist = getlistwordsentence2(tdf['text'])
print(text_wordlist)

def getvectors(wordslist, normalize=True):
    #Answer[[
    
    #]]Answer
    

x, sentences = getvectors(text_wordlist,True)

print(x)

def getSims(x):
    #Answer[[
    
    #]]Answer

innerproducts=getSims(x)

In [ ]:
def visual_similarity_matrix(innerproducts):
  fig, ax = plt.subplots()
  im = ax.imshow(innerproducts)

  # We want to show all ticks...
  ax.set_xticks(np.arange(len(sentences)))
  ax.set_yticks(np.arange(len(sentences)))
  # ... and label them with the respective list entries
  ax.set_xticklabels(sentences)
  ax.set_yticklabels(sentences)

  # Rotate the tick labels and set their alignment.
  plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

  # Loop over data dimensions and create text annotations.
  for i in range(len(sentences)):
    for j in range(len(sentences)):
      text = ax.text(j, i, "%.2f" % innerproducts[i, j],
                       ha="center", va="center", color="w")

  ax.set_title("Produit scalaire des phrases")
  fig.tight_layout()
  fig.set_size_inches(38.5, 38.5)
  plt.show()

visual_similarity_matrix(innerproducts)

## Utilisation des embeddings pour une tâche de classification

On s'intéresse maintenant à apprendre un classifieur sur les données de l'exemple jouet défini ci dessus. On utilisera le jeu de données suivant comme jeu de validation :



In [ ]:
vdf = pd.DataFrame([
    ['the bike drives on the road', 'Y'],
    ['a lion and a cat in a tree', 'N'],
    ['two cars crashed', 'Y'],
    ['i always go to work by bike', 'Y'],
    ['i have no animal at home', 'N'],
    ['dogs like cheese', 'N'], 
    ['a pink flamingo','N'],
    ['trucks','Y'],
    ['truckks','Y'],
    ['truckmegatruck', 'Y'], 
    ['a text about trucks, not animals','Y'], 
    ['a text about animals, not trucks','N'],
    ['doggs','N']
], columns=['text', 'label'])
vdf

La première étape  consiste à transformer les données (label numérique, tokenization, construction d'un batch.)

In [ ]:
import datasets
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(tdf)
validation_dataset = Dataset.from_pandas(vdf)
print(train_dataset,validation_dataset)
print(train_dataset["text"])

# Creation d'un index de padding pour les positions a ignorer par le modèle (e.g., mots inconnus)
pad_idx=embeds.vectors.shape[0]



def preprocess_function(examples):
    vocab=embeds.key_to_index
    inputs= getlistwordsentence2(examples["text"])
    
    inputs = [[(vocab[word] if word in vocab else pad_idx) for word in sentence] for sentence in inputs] 
    
    labels = [(1 if l=='Y' else 0) for l in examples["label"]] 
    ret = {}
    ret["input_ids"]=inputs
    ret["labels"]=labels
    
    return ret


train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.features.keys(), load_from_cache_file=True)
print(train_dataset)
print(train_dataset["input_ids"])
validation_dataset = validation_dataset.map(preprocess_function, batched=True, remove_columns=validation_dataset.features.keys(), load_from_cache_file=True)
print(validation_dataset)
print(validation_dataset["input_ids"])

In [ ]:
from torch.utils.data import DataLoader
from transformers.trainer_pt_utils import LengthGroupedSampler,Sampler,get_length_grouped_indices

batchsize=4
megabatch_mul=16


# Un sampler permet de générer les batchs en selectionnant des indices d'échantillons de manière aléatoire
# dans les données d'entrée
# Ici on souhaite minimiser le padding, donc on choisit de regrouper au maximum les sequences par longueur 
# dans les batchs
# C'est fait selon les étapes suivantes :  
#     1 - Tirage d'une permutation des indices de manière aléatoire
#     2 - Découpage de la liste en megabatchs de taille batch_size*megabatch_mul
#     3 - Tri des éléments par ordre de longueur déscendante à l'intérieur de chaque megabatch
#     4 - Concaténation des listes d'indices retriées localement 
#     5 - Découpage en batch de taille batchsize
#
# Ainsi:
#     - si megabatch_mul trop grand, alors aucun aléatoire (ce qui peu être gênant pour l'apprentissage dans certains cas)
#                                   ==> les batchs seront toujours les mêmes (données triées par longueur, minimisation optimale du padding)
#     - si megabatch_mul trop petit (e.g. = 1), alors batchs complètement aléatoires (séquences simplement ordonnées à l'intérieur du batch, pas d'optimisation sur la minimisation du padding) 
#                                   ==> beaucoup de padding possible (rajoute de la complexité à l'apprentissage)
#
# Il s'agit de trouver un bon compromis, jouer en tp avec les valeurs de megabatch_mul et observer les effets
class LengthGroupedSampler(Sampler):
    r"""
    Sampler that samples indices in a way that groups together features of the dataset of roughly the same length while
    keeping a bit of randomness.
    """

    def __init__(self,batch_size,dataset, megabatch_mul=None):
        self.batch_size = batch_size
        lengths = [len(sample["input_ids"]) for sample in dataset]
        self.lengths = lengths
        self.megabatch_mul=megabatch_mul

    def __len__(self):
        return len(self.lengths)

    def __iter__(self):
        indices = get_length_grouped_indices(self.lengths, self.batch_size, self.megabatch_mul)
        return iter(indices)

    
# Fonction qui produit des tenseurs pytorch a partir d'un batch de données issu du sampler
#
# ici on crée deux tenseurs par batch :
#         - un tenseur pour les labels
#         - un tenseur pour les index de mots des séquences, complétées avec l'idex de padding pad_idx pour les séquences plus courtes (afin d'avoir des données de même taille sur chaque ligne du tenseur, ce qui est requis pour leur création) 
def data_collator(batch):
    first = batch[0]
    ret = {}
    dtype = torch.long if type(first["labels"]) is int else torch.float
    ret["labels"] = torch.tensor([f["labels"] for f in batch], dtype=dtype)
    longest=max([len(l["input_ids"]) for l in batch])
    s = np.stack([np.pad(x["input_ids"], (0, longest - len(x["input_ids"])), constant_values=pad_idx) for x in batch])
    ret["input_ids"] = torch.tensor(s)
    return ret


train_sampler = LengthGroupedSampler(batchsize, train_dataset, megabatch_mul)

train_loader=DataLoader(train_dataset,batchsize,sampler=train_sampler,collate_fn=data_collator,pin_memory=True, shuffle=False, num_workers=0)

for data in train_loader:
    print(data)

test_sampler = LengthGroupedSampler(batchsize, validation_dataset, megabatch_mul)    
    
test_loader=DataLoader(validation_dataset,batchsize,sampler=test_sampler,collate_fn=data_collator,pin_memory=True, shuffle=False, num_workers=0)


for data in test_loader:
    print(data)
    


    


In [ ]:
### On peut créer une fonction de décodage des batchs
def batch_decode(input_ids):
    x=input_ids.data
    x=[" ".join([embeds.index_to_key[i] for i in s if i!=pad_idx]) for s in x]
    return x

for data in test_loader:
    print(data)
    print(batch_decode(data["input_ids"]))



On note que le dernier texte "truckmegatruck" est décodé sous la chaîne vide: il ne contient que des mots inconnus, donc ignorés... 

### Modèle de classifcation simple

On construit ensuite le modèle de classification avec torch. La couche d'entrée correspond aux embeddings, implémentés par nn.Embedding(). nn.Embedding() permet de construire la matrice d'embeddings sur l'ensemble du vocabulaire. Ils sont ensuite "activés" en fonction du texte d'entrée. Le modèle à construire fait une simple moyenne des embeddings des mots des textes, applique une activation tanh et envoie le resultat à travers un Linear à deux sorties. Penser à indiquer l'index de padding à la construction de nn.Embedding, car çà permet de ne pas les prendre en compte dans les calculs de gradients (et de conserver ces embeddings vides à zeros pour ne pas en dépendre => invariance par rapport à la longueur). Penser aussi à ajouter de la l2 sur les embeddings (très importants pour que les représentations ne s'éparpillent pas aux confins de l'espace de représentation!). On pourra aussi ajouter du Dropout. 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

        
class MyModel(nn.Module):
    #Answer[[
    
    #]]Answer

On entraîne et teste le modèle.

In [ ]:
def train_test(train_iter, test_iter, model, loss_function, optimizer, epochs, clip=-1):
  for epoch in range(epochs):
      epoch_loss = 0
      epoch_accuracy = 0
      model.train()
      nb_samples=0
      for batch in train_iter:
          optimizer.zero_grad()
          #print("text shape ",batch.text.T.shape)
          prediction = model(batch["input_ids"])
          if not isinstance(prediction,torch.Tensor):  
                prediction = prediction["logits"]
          #print(prediction)
          loss = loss_function(prediction, batch["labels"])

          loss.backward()
          if clip>0:
              torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
          optimizer.step()
          nb_samples+=prediction.shape[0]
          epoch_loss+=loss.item()*prediction.shape[0]
          preds=(prediction[:,1]>prediction[:,0])*1.0
          accuracy=(preds==batch["labels"]).sum()
          epoch_accuracy+=accuracy.item()
      print('train loss on epoch {} : {:.3f}'.format(epoch, epoch_loss/nb_samples))
      print('train accuracy on epoch {}: {:.3f}'.format(epoch, epoch_accuracy/nb_samples))
      
      model.eval()
      test_loss = 0
      test_accuracy = 0
      nb_samples=0
      accuracy=0
      for batch in test_iter:
          #print("test ",batch)
          with torch.no_grad():
              optimizer.zero_grad()
                
              prediction = model(batch["input_ids"])
              if not isinstance(prediction,torch.Tensor):
                    prediction = prediction["logits"]
              loss = loss_function(prediction, batch["labels"])
              nb_samples+=prediction.shape[0]
              test_loss+=loss.item()*prediction.shape[0]
              #print(batch_decode(batch["input_ids"]))
              preds=(prediction[:,1]>prediction[:,0])*1.0
              accuracy=(preds==batch["labels"]).sum()
              test_accuracy+=accuracy.item()  
              #print(preds,accuracy)
      print('test loss on epoch {}: {:.3f}'.format(epoch, test_loss/nb_samples))
      print('test accuracy on epoch {}: {:.3f}'.format(epoch, test_accuracy/nb_samples))



In [ ]:
# penser à ajouter un embedding pour pad_idx        
 
#[[Answer 
# net = ...

#/Answer]]

loss_function = nn.CrossEntropyLoss()
optimizer = Adam(net.parameters(), lr=0.01,weight_decay=0.01)
epochs = 5000
train_test(train_loader, test_loader, net, loss_function, optimizer, epochs)

In [ ]:
def predict(sentence,model):
    examples={}
    examples["text"]=[sentence]
    examples["label"]=["Y"]
    data=preprocess_function(examples)
    #print(data)
    data=data_collator([data])
    #print(data["input_ids"])
    model.eval()
    with torch.no_grad():
        prediction = model(data["input_ids"][0])
        if not isinstance(prediction,torch.Tensor):
            prediction = prediction["logits"]
        preds=(prediction[:,1]>prediction[:,0])*1.0
    return {"logits":prediction,"prediction":preds}
    
def predict_from_pandas(datap,net):
    data=Dataset.from_pandas(vdf)
    accuracy=0
    size=len(data["text"])
    for i in range(size):
        s=data["text"][i]
        l=data["label"][i]
        l=(1 if l=='Y' else 0)
        p=predict(s,net)
        print(s,p, "truth=",l)
        accuracy+=(p["prediction"]==l)
    return accuracy/size

print(predict_from_pandas(vdf,net))
    

On observe beaucoup de sur-apprentissage. Normal étant donné la taille du corpus d'apprentissage...


### Même modèle mais avec des embeddings pré-entraînés

 Voyons ce que celà donne avec les représentations pre-entrainées (que l'on freeze) 

#### Méthode 1 : directement fournir les embeddings construits à partir du vocab

In [ ]:

#[[Answer 
# net = ...
# weights=

#/Answer]]

net.embedding.weight.data=weights   # on charge les pre-train 
net.embedding.weight.requires_grad=False  # on freeze
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(net.parameters(), lr=0.01,weight_decay=0.0)
epochs = 5000

train_test(train_loader, test_loader, net, loss_function, optimizer, epochs,clip=1)


#### Méthode 2 : avec la méthode nn.Embedding.from_pretrained()

nn.Embedding permet d'importer directement une matrice de poids (embeddings pré-entraînés) grâce à la méthode from_pretrained() :  
```
weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
embedding = nn.Embedding.from_pretrained(weight)
```
Plus particulièrement, la matrice de poids correspond à notre élément vocab.vectors qui va permettre d'initialiser la matrice d'embeddings.

Par défaut, les embeddings sont "gelés" : ils ne sont pas modifiés avec la backpropagation, mais il est possible de les modifier avec le paramètre freeze=False. Cela revient à "fine-tuner" les embeddings sur la tâche.

In [ ]:
#[[Answer 

#/Answer]]

loss_function = nn.CrossEntropyLoss()
optimizer = Adam(net.parameters(), lr=0.01,weight_decay=0.0)
epochs = 5000
train_test(train_loader, test_loader, net, loss_function, optimizer, epochs)



In [ ]:
print(predict_from_pandas(vdf,net))

Ok c'est beaucoup mieux, mais on a toujours des limites importantes: 
- Pas de prise en compte de l'ordre des mots : on voit que "a text about trucks, not animals" et "a text about animals, not trucks" retournent exactement les mêmes scores de prediction
- Pas de gestion des mots hors vocabulaire. Exemple truckmegatruck n'est pas géré, et retourne exactement les mêmes predictions que les deux textes avec fautes de frappe truckks et doggs 

###  Tokenizers 

Pour aller plus loin, on propose maintenant d'utiliser des tokens issus d'un tokenizer plus évolué, du type Byte Pair Encoding vu en cours, pour voir si cela pourrait améliorer les performances. 

On commence par récupérer un tokenizer pré-entraîné sur un corpus : 

In [ ]:

from transformers import AutoTokenizer, DistilBertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# De la même manière qu'avec Glove, on peut observer le vocabulaire : 
print(tokenizer.vocab)

vocab_size=len(tokenizer.vocab)
print("Size of Vocab : ",vocab_size)


Pour pouvoir utiliser ce tokenizer dans nos modèles, il faut recréer le datasets. 

Donner ci-dessous le code de la fonction preprocess_function à utiliser maintenant :

In [ ]:
# On recrée nos datasets initiaux que l'on va traiter différemment : 
train_dataset = Dataset.from_pandas(tdf)
validation_dataset = Dataset.from_pandas(vdf)





def preprocess_function(examples):
#[[Answer
    
#/Answer]]

train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.features.keys(), load_from_cache_file=True)
print(train_dataset)
print(train_dataset["input_ids"])
validation_dataset = validation_dataset.map(preprocess_function, batched=True, remove_columns=validation_dataset.features.keys(), load_from_cache_file=True)
print(validation_dataset)
print(validation_dataset["input_ids"])

In [ ]:
pad_idx=tokenizer.pad_token_id  # On met à jour l'index de padding et on recrée les dataloaders

train_sampler = LengthGroupedSampler(batchsize, train_dataset, megabatch_mul)

train_loader=DataLoader(train_dataset,batchsize,sampler=train_sampler,collate_fn=data_collator,pin_memory=True, shuffle=False, num_workers=0)

for data in train_loader:
    print(data)
    print(tokenizer.batch_decode(data["input_ids"]))

test_sampler = LengthGroupedSampler(batchsize, validation_dataset, megabatch_mul)    
    
test_loader=DataLoader(validation_dataset,batchsize,sampler=test_sampler,collate_fn=data_collator,pin_memory=True, shuffle=False, num_workers=0)


for data in test_loader:
    print(data)
    print(tokenizer.batch_decode(data["input_ids"]))
    


On note que truckmegatruck n'est plus complètement ignoré mais découpé en tokens (on retrouve par exemple le token truck d'id 4744 au début du mot). 

On a donc un nouvel encodage de notre corpus, mais nous ne possédons pas d'embeddings pour les tokens correspondants. On peut essayer d'en récupérer en chargeant un modèle transformer pre-entraîné : 

In [ ]:
distil_bert = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
print(distil_bert)

On se propose d'extraire les embeddings de ce modèle pour les utiliser directement dans notre modèle simple MyModel défini plus haut. Donner la procédure pour charger ces embeddings et lancer l'entraînement du modèle. 

In [ ]:

#[[Answer

#/Answer]]

loss_function = nn.CrossEntropyLoss()
optimizer = Adam(net.parameters(), lr=0.01,weight_decay=0.000)
epochs = 50000

train_test(train_loader, test_loader, net, loss_function, optimizer, epochs) 


In [ ]:
print(predict_from_pandas(vdf,net))

Ok pas vraiment mieux qu'avec les embeddings de mots (et même plutôt moins bien!). Pourquoi d'après vous ? 

\#[[Answer



\#/Answer]]

# Adaptation d'un modèle pre-entraîné : DistillBert

On va plutôt essayer de re-utiliser le modèle DistillBert complet, en ne fine-tunant que la dernière couche linéaire. Donner la procédure pour réaliser cela et lancer l'entraînement. 

In [ ]:

#ANSWER[[

#/ANSWER]]

In [ ]:
print(predict_from_pandas(vdf,net))

Ok... on sent qu'avec beaucoup plus de données on pourrait arriver à une très bonne qualité mais le dataset d'apprentissage est vraiment trop petit pour un modèle aussi gros.

# Module Transformer

Essayons maintenant de créer un petit réseau Transformer d'un seul layer d'encoder, qui reutilise les embeddings de DistilBert (seulement ceux des mots dans un premier temps), avec 8 têtes de self-attention. Attention, par défaut la taille des séquences est la première dimension des entrées attendues par le module TransformerEncoderLayer. Pour donner plutôt sous une forme (taille du batch, longueur des sequences, taille des embeddings), utiliser l'option batch_first=True. Le token CLS à utiliser pour faire la classification est le premier de chaque sequence (c'est sur lui qu'on branche la tête de classification, qui correspond à une simple couche linéaire, précédée d'une activation tanh) 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

        
class TransfoModel(nn.Module):
    #Answer[[
    
    #]]Answer
    
embeds=distil_bert.distilbert.embeddings.word_embeddings

net = TransfoModel(vocab_size,embeds.weight.shape[-1],pad_idx,0.5,1)
net.embedding=nn.Embedding.from_pretrained(embeds.weight)   # on charge les pre-train (en supposant que le module Embedding du modèle soit dans une variable embeddings)

loss_function = nn.CrossEntropyLoss()
optimizer = Adam(net.parameters(), lr=0.001,weight_decay=0.0)
epochs = 50000

train_test(train_loader, test_loader, net, loss_function, optimizer, epochs, clip=1)


Ca a l'air mieux ! Voyons les predictions sur nos exemples précédents : 

In [ ]:
print(predict_from_pandas(vdf,net))

Ok, les fautes de frappes ont l'air mieux gérées. Le mot complexe également.. 
Par contre on a toujours la limite de l'ordre, avec les deux phrases "a text about trucks, not animals" et "a text about animals, not trucks" retournant exactemenent les mêmes prédictions. Normal, un transformer est de base invariant à l'ordre ! 

# Positionnal Embeddings



In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
#from sentence_transformers import SentenceTransformer
        
class TransfoModel(nn.Module):
    #Answer[[
    
    #]]Answer
    
embeds=distil_bert.distilbert.embeddings.word_embeddings

net = TransfoModel(vocab_size,embeds.weight.shape[-1],pad_idx,0.2,1)
net.embedding=nn.Embedding.from_pretrained(embeds.weight)   # on charge les pre-train (en supposant que le module Embedding du modèle soit dans une variable embeddings)
net.pos_embedding=nn.Embedding.from_pretrained(distil_bert.distilbert.embeddings.position_embeddings.weight)
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(net.parameters(), lr=0.001,weight_decay=0.0)
epochs = 50000

train_test(train_loader, test_loader, net, loss_function, optimizer, epochs,clip=0.1)



In [ ]:
print(predict_from_pandas(vdf,net))

Super ! Bon çà reste des resultats fragiles vue la taille du jeu de données, mais on semble quand même obtenir une capacité de traitement bien supérieure aux modèles précédents. On note en particulier la capacité à distinguer les phrases possédant des mots identiques mais dans un ordre différent. 

# Experimentations sur un Jeu de Données Réel : IMDB

Pour terminer ce TP, on s'intéresse à l'adaptation du modèle DistillBert utilisé ci-dessus, sur un jeu de données  beaucoup plus conséquent: le corpus IMDB (base de données de commentaires sur des films). Pour cette partie, il est largement conseillé de travailler sur GPU. 

In [ ]:
import datasets
dataset = datasets.load_dataset("imdb")

In [ ]:

print(dataset)

In [ ]:
train_fraction=0.2            
train_dataset=dataset["train"]
train_dataset = train_dataset.train_test_split(test_size=1-train_fraction)["train"]
test_fraction=0.002
validation_dataset=dataset["test"]
validation_dataset = validation_dataset.train_test_split(test_size=1-test_fraction)["train"]

from transformers import AutoTokenizer, DistilBertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
#[[Answer
    
#/Answer]]


train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.features.keys(), load_from_cache_file=True)
print(train_dataset)

validation_dataset = validation_dataset.map(preprocess_function, batched=True, remove_columns=validation_dataset.features.keys(), load_from_cache_file=True)
print(validation_dataset)




In [ ]:
pad_idx=tokenizer.pad_token_id  # On met à jour l'index de padding et on recrée les dataloaders

batchsize=32
megamul=4

train_sampler = LengthGroupedSampler(batchsize, train_dataset, megabatch_mul)

train_loader=DataLoader(train_dataset,batchsize,sampler=train_sampler,collate_fn=data_collator,pin_memory=True, shuffle=False, num_workers=0)

for data in train_loader:
    print(data)
    print(tokenizer.batch_decode(data["input_ids"]))
    break

test_sampler = LengthGroupedSampler(batchsize, validation_dataset, megabatch_mul)    
    
test_loader=DataLoader(validation_dataset,batchsize,sampler=test_sampler,collate_fn=data_collator,pin_memory=True, shuffle=False, num_workers=0)


for data in test_loader:
    print(data)
    print(tokenizer.batch_decode(data["input_ids"]))
    break

In [ ]:
def train_test(trainloader, testloader, model, loss_function, optimizer, epochs, clip=-1, test_rate=1):
  it=0

  

  for epoch in range(epochs):
      
      
      epoch_loss = 0
      epoch_accuracy = 0
      
     
    
      nb_samples=0
      t = tqdm(iter(trainloader), total=len(trainloader), dynamic_ncols=True, position=0)
      train_loss_log = tqdm(total=0, position=4, bar_format='{desc}')
      test_log = tqdm(total=0, position=2, bar_format='{desc}')
      accuracy_log = tqdm(total=0, position=3, bar_format='{desc}')
      for batch in t:
          it+=1
          if it%test_rate==0:
            test(testloader, model, loss_function, epoch,(test_log,accuracy_log))
          model.train()
          optimizer.zero_grad()
          #print("text shape ",batch.text.T.shape)
          prediction = model(batch["input_ids"].to(model.device))
          if not isinstance(prediction,torch.Tensor):  
                prediction = prediction["logits"]
          #print(prediction)
          loss = loss_function(prediction, batch["labels"].to(model.device))
          train_loss_log.set_description_str("loss train {:.3f} ".format(loss.item()))
          loss.backward()
          if clip>0:
              torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
          optimizer.step()
          nb_samples+=prediction.shape[0]
          epoch_loss+=loss.item()*prediction.shape[0]
          preds=(prediction[:,1]>prediction[:,0])*1.0
          accuracy=(preds==batch["labels"].to(model.device)).sum()
          epoch_accuracy+=accuracy.item()
      print('train loss on epoch {} : {:.3f}'.format(epoch, epoch_loss/nb_samples))
      print('train accuracy on epoch {}: {:.3f}'.format(epoch, epoch_accuracy/nb_samples))
      
def test(testloader, model, loss_function, epoch, descs):
      model.eval()
      test_loss = 0
      test_accuracy = 0
      nb_samples=0
      accuracy=0
      t = tqdm(iter(testloader), total=len(testloader), position=1, leave=False)
      tl,al=descs
      #test_log = tqdm(total=0, position=2, bar_format='{desc}')
      #accuracy_log = tqdm(total=0, position=3, bar_format='{desc}')
      for batch in t:
          #print("test ",batch)
          with torch.no_grad():
              optimizer.zero_grad()
              inputs=batch["input_ids"].to(model.device)
              prediction = model(inputs)
              if not isinstance(prediction,torch.Tensor):
                    prediction = prediction["logits"]
              loss = loss_function(prediction, batch["labels"].to(model.device))
              nb_samples+=prediction.shape[0]
              test_loss+=loss.item()*prediction.shape[0]
              #print(batch_decode(batch["input_ids"]))
              preds=(prediction[:,1]>prediction[:,0])*1.0
              accuracy=(preds==batch["labels"].to(model.device)).sum()
              test_accuracy+=accuracy.item()  
              #print(preds,accuracy)
      tl.set_description_str('test loss on epoch {}: {:.3f}'.format(epoch, test_loss/nb_samples))
      al.set_description_str('test accuracy on epoch {}: {:.3f}'.format(epoch, test_accuracy/nb_samples))
      #print('test loss on epoch {}: {:.3f}'.format(epoch, test_loss/nb_samples))
      #print('test accuracy on epoch {}: {:.3f}'.format(epoch, test_accuracy/nb_samples))
      

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

distil_bert = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
net = distil_bert
def set_parameter_requires_grad(module, requires_grad=True):
    for param in module.parameters():
        param.requires_grad = requires_grad


net=net.to(device)

# Fixons d'abord les poids du réseau :
set_parameter_requires_grad(net,False)
#set_parameter_requires_grad(net.distilbert.embeddings,False)
set_parameter_requires_grad(net.classifier,True)

params_to_update = []
for name,param in net.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(params_to_update, lr=0.0001,weight_decay=0.000)
epochs = 50000


train_test(train_loader, test_loader, net, loss_function, optimizer, epochs, test_rate=10)
